In [2]:
%matplotlib widget
import lppydsmc.poisson_solver as ps
import numpy as np
from fenics import * # for plotting
# from plotting import analysis
import os
os.environ["MKL_NUM_THREADS"] = "1" 
os.environ["NUMEXPR_NUM_THREADS"] = "1" 
os.environ["OMP_NUM_THREADS"] = "1" 

# A first example

In [3]:
in_vertices = np.array([[0,0],[1,0],[1,1],[0,1]]) 
out_vertices_list = [np.array([[0.25, 0.5],[0.5,0.25],[0.75,0.5],[0.5,0.75]])] # list of all polygones to remove
mesh = ps.mesh.polygonal(resolution = 10, in_vertices = in_vertices, out_vertices_list = out_vertices_list)

In [4]:
# poisson solver 

# boundary conditions
boundary_conditions = { 
                'top' : {
                    'type' : 'Dirichlet',
                    'solution' : '1 + x[0]*x[0] + 2*x[1]*x[1]', # 2D, x[1] = 'x', x[2] = 'y' for example.
                    'solution_kwargs' : {},
                    'degree' : 2,
                    'boundary' : 'on_boundary && near(x[1], 1, tol)', # must return a boolean
                                                                      # on_boundary corresponds to a boolean fenics returns, because it knows in practice if *x* is on the boundary or not.
                                                                      # *near(x[0], 1, tol)* is equivalent to *abs(x[0]-1)<tol*
                    'boundary_kwargs' : {
                        'tol' : 1e-14
                    }               
                },
                'bottom' : {
                    'type' : 'Dirichlet',
                    'solution' : 'x[0] <= 0.5 + tol ? k_0 : k_1',
                    'solution_kwargs' : {
                        'k_0' : 1.0,
                        'k_1' : 0.01,
                        'tol' : 1e-14
                    },
                    'degree' : 0,
                    'boundary' : 'on_boundary && near(x[1], 0, tol)',
                    'boundary_kwargs' : {
                        'tol' : 1e-14
                    }
                }
            }
# charge density
charge_density = {
                'value' : 'max_density-x[0]*(max_density-min_density)/lx',  # must be a string too
                'degree' : 2,
                'kwargs' : {
                    'max_density' : 2e17,
                    'min_density' : 1e17,
                    'lx' : 1.0 # size of the system - no offset
                } 
            }

In [5]:
potential_field, electric_field = ps.solver(mesh, boundary_conditions, charge_density) 

In [6]:
import matplotlib.pyplot as plt

E = electric_field
Phi = potential_field

In [7]:
Ex, Ey = E.split(deepcopy=True)
NE=sqrt(dot(E,E))

plt.figure(figsize=(5,10), constrained_layout = True)

plt.subplot(2,1,1)
fig=plot(Phi)
plt.title('U(V)', size=10)
fig.set_cmap("viridis") 
plt.colorbar(fig, shrink=1)
plt.xlabel('x(m)', size=10)
plt.ylabel('y(m)', size=10)

plt.subplot(2,1,2)
fig=plot(NE)
plt.title('E(V/m)', size=10)
fig.set_cmap("viridis") 
plt.colorbar(fig)
plt.xlabel('x(m)', size=10)
plt.ylabel('y(m)', size=10)

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Can we reproduce previous results on the thruster ?

In [8]:
import lppydsmc as ld
dp = 0.001  

# here, x : l, y : w
dict_thruster = {
    'w_in' : 5*dp,
    'l_in' : 3*dp,
    'w_1' : 3*dp,
    'l_1' : dp,
    'l_int' : dp,
    'w_2' : dp,
    'l_2' : 10*dp,
    'w_out' : 5*dp,
    'l_out' : dp,
    'offsets' : np.array([0,0]) 
}
# Note:  l_mot = l_in 
in_vertices = np.flip(ld.systems.helper.thruster_points(**dict_thruster), axis = 0)
mesh = ps.mesh.polygonal(resolution = 100, in_vertices = in_vertices)

In [9]:
tol = 1e-14

# useful lenghts to define boundary conditions
x_in = dict_thruster['offsets'][0]
x_electrode_1 = x_in + dict_thruster['l_in']
x_inter_electrodes_area = x_electrode_1 + dict_thruster['l_1']
x_electrode_2 = x_inter_electrodes_area + dict_thruster['l_int']
x_out = x_electrode_2 + dict_thruster['l_2']

In [16]:
# poisson solver 

potential_boundary_conditions = {
    'inflow_area' : '0.0',
    'inflow_area_sides': 'Neumann',
    'electrode_1' : '30.0',
    'inter_electrode_area':'Neumann',
    'electrode_2': '300.0'
}

# boundary conditions
boundary_conditions = {
                'inflow_area' : {
                    'type' : 'Dirichlet',
                    'solution' : potential_boundary_conditions['inflow_area'], 
                    'solution_kwargs' : {},
                    'degree' : 0, # because constant
                    'boundary' : 'on_boundary && near(x[0], x_in, tol)', 
                    'boundary_kwargs' : {
                        'tol' : tol,
                        'x_in' : x_in
                    }               
                },
                'inflow_area_sides' : {
                    'type' : 'Neumann'
                },
                'electrode_1' : {
                    'type' : 'Dirichlet',
                    'solution' : potential_boundary_conditions['electrode_1'], 
                    'solution_kwargs' : {},
                    'degree' : 0,
                    'boundary' : 'on_boundary && x[0] > x_electrode_1 - tol && x[0] < x_inter_electrodes_area + tol',
                    'boundary_kwargs' : {
                        'tol' : tol,
                        'x_electrode_1' : x_electrode_1,
                        'x_inter_electrodes_area' : x_inter_electrodes_area
                    }
                },
                'inter_electrode_area' : {
                    'type' : 'Neumann'
                },
                'electrode_2' : {
                    'type' : 'Dirichlet',
                    'solution' : potential_boundary_conditions['electrode_2'], 
                    'solution_kwargs' : {},
                    'degree' : 0,
                    'boundary' : 'on_boundary && x[0] > x_electrode_2 - tol && x[0] < x_out + tol',
                    'boundary_kwargs' : {
                        'tol' : tol,
                        'x_electrode_2' : x_electrode_2,
                        'x_out' : x_out
                    }
                }
            }
# charge density
charge_density = {
                'value' : '-q*n0*(1-x[0]/l)',  # must be a string too
                'degree' : 2,
                'kwargs' : {
                    'n0':1e17,
                    'l':0.016,
                    'q':1.6e-19,
                }
            }

charge_density_empty = {
                'value' : '0',  # must be a string too
                'degree' : 0,
                'kwargs' : {}
            }

In [17]:
potential_field, electric_field = ps.solver(mesh, boundary_conditions, charge_density) 
potential_field_2, electric_field_2 = ps.solver(mesh, boundary_conditions, charge_density_empty) 

In [12]:
import matplotlib.pyplot as plt

In [18]:
dPhi = potential_field_2-potential_field
dE = electric_field_2-electric_field

In [19]:
# Ex, Ey = E.split(deepcopy=True)
dNE=sqrt(dot(dE,dE))

plt.figure(figsize=(10,10), constrained_layout = True)

plt.subplot(1,1,1)
fig=plot(dPhi)
plt.title('dU(V)', size=30)
fig.set_cmap("viridis") 
plt.colorbar(fig, shrink=1)
plt.xlabel('x(m)', size=20)
plt.ylabel('y(m)', size=20)

# plt.subplot(2,1,2)
# fig=plot(dNE)
# plt.title('dE(V/m)', size=30)
# fig.set_cmap("viridis") 
# plt.colorbar(fig)
# plt.xlabel('x(m)', size=20)
# plt.ylabel('y(m)', size=20)

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Calling FFC just-in-time (JIT) compiler, this may take some time.


/home/paul/anaconda3/envs/NIRN/lib/python3.7/site-packages/ffc/uflacs/analysis/dependencies.py:61: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  active[targets] = 1


In [20]:
n = 1e15
q = -1.6e-19
rho = n*q

l = 1e-3
Vl = 30 # V
V0 = 30
eps0 = 8.85e-12

def potential(x, rho, l, Vl, V0):
    return -rho*x/(2*eps0)*(x-l)+(Vl-V0)*(x/l)+V0 # rho/(2*eps0)*x*x+(V/l-rho*l/2*eps0)*x


In [21]:
sol0 = lambda x: potential(x, 0, l, Vl, V0)
sol1 = lambda x: potential(x, q*1e14, l, Vl, V0)
sol2 = lambda x: potential(x, q*1e15, l, Vl, V0)
sol3 = lambda x: potential(x, q*1e16, l, Vl, V0)
sol4 = lambda x: potential(x, q*1e17, l, Vl, V0)

In [22]:
x = np.linspace(0,l,100)
fig, ax = plt.subplots()
s0 = sol0(x)
# ax.plot(x, sol0(x), label = '$n = 0$ $m^{-3}$')
ax.plot(np.abs((sol1(x)-s0)/s0), x, label = '$n = 10^{14}$ $m^{-3}$')
ax.plot(np.abs((sol2(x)-s0)/s0), x, label = '$n = 10^{15}$ $m^{-3}$')
ax.plot(np.abs((sol3(x)-s0)/s0), x, label = '$n = 10^{16}$ $m^{-3}$')
ax.plot(np.abs((sol4(x)-s0)/s0), x, label = '$n = 10^{17}$ $m^{-3}$')
ax.set_xscale('log')
ax.legend(loc='best')
ax.set_xlabel('relative difference to no charge density')
ax.set_ylabel('y (m)')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'y (m)')

# Thruster 3 grids

In [14]:
import lppydsmc as ld
dp = 0.001  

# here, x : l, y : w
dict_thruster = {
    'w_in' : 5*dp,
    'l_in' : 3*dp,
    'w_1' : 3*dp,
    'l_1' : dp,
    'l_int' : dp,
    'w_2' : dp,
    'l_2' : 10*dp,
    'l_int_2' : dp,
    'w_3' : 3*dp,
    'l_3' : 1*dp,
    'w_out' : 5*dp,
    'l_out' : dp,
    'offsets' : np.array([0,0]) 
}
# Note:  l_mot = l_in 
in_vertices = np.flip(ld.systems.helper.thruster_three_grids_points(**dict_thruster), axis = 0)
mesh = ps.mesh.polygonal(resolution = 100, in_vertices = in_vertices)

In [15]:
tol = 1e-14

# useful lenghts to define boundary conditions
x_in = dict_thruster['offsets'][0]
x_electrode_1 = x_in + dict_thruster['l_in']
x_inter_electrodes_area = x_electrode_1 + dict_thruster['l_1']
x_electrode_2 = x_inter_electrodes_area + dict_thruster['l_int']
x_inter_electrodes_area_2 = x_electrode_2 + dict_thruster['l_2']
x_electrode_3 = x_inter_electrodes_area_2 + dict_thruster['l_int_2']
x_out = x_electrode_3 + dict_thruster['l_3']

In [16]:
# poisson solver 

potential_boundary_conditions = {
    'inflow_area' : '0.0',
    'inflow_area_sides': 'Neumann',
    'electrode_1' : '30.0',
    'inter_electrode_area':'Neumann',
    'electrode_2': '300.0',
    'inter_electrode_area_2':'Neumann',
    'electrode_3': '-100.0'
}

# boundary conditions
boundary_conditions = {
                'inflow_area' : {
                    'type' : 'Dirichlet',
                    'solution' : potential_boundary_conditions['inflow_area'], 
                    'solution_kwargs' : {},
                    'degree' : 0, # because constant
                    'boundary' : 'on_boundary && near(x[0], x_in, tol)', 
                    'boundary_kwargs' : {
                        'tol' : tol,
                        'x_in' : x_in
                    }               
                },
                'inflow_area_sides' : {
                    'type' : 'Neumann'
                },
                'electrode_1' : {
                    'type' : 'Dirichlet',
                    'solution' : potential_boundary_conditions['electrode_1'], 
                    'solution_kwargs' : {},
                    'degree' : 0,
                    'boundary' : 'on_boundary && x[0] > x_electrode_1 - tol && x[0] < x_inter_electrodes_area + tol',
                    'boundary_kwargs' : {
                        'tol' : tol,
                        'x_electrode_1' : x_electrode_1,
                        'x_inter_electrodes_area' : x_inter_electrodes_area
                    }
                },
                'inter_electrode_area' : {
                    'type' : 'Neumann'
                },
                'electrode_2' : {
                    'type' : 'Dirichlet',
                    'solution' : potential_boundary_conditions['electrode_2'], 
                    'solution_kwargs' : {},
                    'degree' : 0,
                    'boundary' : 'on_boundary && x[0] > x_electrode_2 - tol && x[0] < x_inter_electrodes_area_2 + tol',
                    'boundary_kwargs' : {
                        'tol' : tol,
                        'x_electrode_2' : x_electrode_2,
                        'x_inter_electrodes_area_2' : x_inter_electrodes_area_2
                    }
                },
                'inter_electrode_area_2' : {
                    'type' : 'Neumann'
                },
                'electrode_3' : {
                    'type' : 'Dirichlet',
                    'solution' : potential_boundary_conditions['electrode_3'], 
                    'solution_kwargs' : {},
                    'degree' : 0,
                    'boundary' : 'on_boundary && x[0] > x_electrode_3 - tol && x[0] < x_out + tol',
                    'boundary_kwargs' : {
                        'tol' : tol,
                        'x_electrode_3' : x_electrode_3,
                        'x_out' : x_out
                    }
                },
            }
# charge density
charge_density = {
                'value' : '0',  # must be a string too
                'degree' : 0,
                'kwargs' : {}
            }

In [17]:
potential_field, electric_field = ps.solver(mesh, boundary_conditions, charge_density) 

In [18]:
import matplotlib.pyplot as plt

E = electric_field
Phi = potential_field

In [21]:
Ex, Ey = E.split(deepcopy=True)
NE=sqrt(dot(E,E))

plt.figure(figsize=(10,10), constrained_layout = True)

plt.subplot(2,1,1)
fig=plot(Phi)
plt.title('U(V)', size=20)
fig.set_cmap("viridis") 
plt.colorbar(fig)
plt.xlabel('x(m)', size=20)
plt.ylabel('y(m)', size=20)

plt.subplot(2,1,2)
fig=plot(NE)
plt.title('E(V/m)', size=20)
fig.set_cmap("viridis") 
plt.colorbar(fig, shrink = )
plt.xlabel('x(m)', size=20)
plt.ylabel('y(m)', size=20)

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
X = np.linspace(0,1.8e-2, 1000)
central_potential_field = np.array([potential_field(x, 2.5e-3) for x in X])

In [9]:
fig, ax = plt.subplots()
ax.plot(X, central_potential_field)
analysis.set_axis(ax, x = 'x', y = 'potential');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Child-Langmuir law

$\frac{d^2\Phi}{dx^2} = -\frac{J_i}{\epsilon_0} (-\frac{2e\Phi(x)}{M})^{-1/2}$

with : $J_i = en_i \sqrt{\frac{kT_i}{2\pi M}}$

In [38]:
e = 1.6e-19 # C
k = 1.38e-23 # J/K/mol 
ni,Ti,M = 3.2e17, 1000, 2.17e-25 # m-3, K, kg
eps0 = 8.82e-12 
def ion_current_density(ni,Ti,M):
    return e*ni*np.sqrt(k*Ti/(2*np.pi*M))

ji = ion_current_density(ni,Ti,M)
fact = -ji/eps0*np.sqrt(2*e/M)

def y_second(y, x):
    return fact*np.sqrt(np.abs(y)) # could add a np.abs

def Y_prime(Y,x): # Y is (y,y')
    return np.array([Y[1], y_second(Y[0],x)])

In [39]:
# integrator 
from scipy.integrate import odeint

In [47]:
sol = odeint(Y_prime, y0 = np.array([30,10]), t = np.linspace(0,5e-4,50))

In [51]:
fig, ax = plt.subplots()
ax.plot(sol[:,1], np.linspace(0,5e-4,50))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …